<a href="https://colab.research.google.com/github/omoreva/kaggle_m5/blob/master/10_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tsfresh

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime  
from datetime import timedelta  
import gc
import pickle
from sklearn import preprocessing, metrics
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import EfficientFCParameters, MinimalFCParameters, ComprehensiveFCParameters

In [4]:
print('Reading files...')
#data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'calendar.csv')
data_folder = '/content/drive/My Drive/Kaggle_M5/01_preprocessed_data/'
X_train = pd.read_pickle(data_folder + 'X_train_WI_2.pkl')
X_val = pd.read_pickle(data_folder + 'X_val_WI_2.pkl')
y_train = X_train['demand']
y_val = X_val['demand']


Reading files...


In [5]:
X_train.head()

id  ... cheaper_than_usual
439578  HOBBIES_1_008_WI_2_validation  ...               True
439579  HOBBIES_1_009_WI_2_validation  ...               True
439580  HOBBIES_1_010_WI_2_validation  ...              False
439581  HOBBIES_1_012_WI_2_validation  ...              False
439583  HOBBIES_1_016_WI_2_validation  ...              False

[5 rows x 46 columns]

In [0]:
my_parameters = {
 'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
  {'f_agg': 'median', 'maxlag': 40},
  {'f_agg': 'var', 'maxlag': 40}],
  'ar_coefficient': [{'coeff': 3, 'k': 10}],
   'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
  {'num_segments': 10, 'segment_focus': 1},
  {'num_segments': 10, 'segment_focus': 2},
  {'num_segments': 10, 'segment_focus': 3},
  {'num_segments': 10, 'segment_focus': 4},
  {'num_segments': 10, 'segment_focus': 5},
  {'num_segments': 10, 'segment_focus': 6},
  {'num_segments': 10, 'segment_focus': 7},
  {'num_segments': 10, 'segment_focus': 8},
  {'num_segments': 10, 'segment_focus': 9}],
 'fft_coefficient': [{'attr': 'angle', 'coeff': 1},
],
}

In [0]:
store_demand = X_train[['item_id', 'date', 'demand']]

In [8]:
store_demand_fe = extract_features(store_demand, 
                     column_id='item_id', column_sort='date',
                     default_fc_parameters=my_parameters)

Feature Extraction: 100%|██████████| 5/5 [00:17<00:00,  3.53s/it]


In [0]:
store_demand_fe = store_demand_fe.reset_index()
store_demand_fe = store_demand_fe.rename(columns={"id": "item_id"})
store_demand_fe.head()

X_train = X_train.merge(store_demand_fe, on = ['item_id'], how = 'left')
X_val = X_val.merge(store_demand_fe, on = ['item_id'], how = 'left')



In [0]:
not_features = ['d', 'id', 'item_id', 'store_id', 'dept_id',
                'state_id',  'date', 'start_date', 
                'snap_CA', 	'snap_TX', 'weekday',
                'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
                'demand_type']

                not_features = ['d', 'id', 'item_id', 'store_id', 'demand', 'date', 'start_date',
                'demand_type', 'event_name_1', 	'event_type_1', 'event_name_2',
                'event_type_2', 'weekday' ]
features = X_train.columns[~X_train.columns.isin(not_features) ]
X_train = X_train[features]
X_val = X_val[features]


In [11]:
features

Index(['cat_id', 'demand', 'month', 'year', 'snap_WI', 'days_from_start',
       'start_date_from_start', 'sell_price', 'lag_28', 'lag_29', 'lag_30',
       'lag_31', 'lag_32', 'lag_33', 'lag_34', 'lag_35', 'rmean_28_3',
       'rmean_28_7', 'ADI', 'CV2', 'avg_weekday_demand', 'avg_demand',
       'max_demand', 'std_demand', 'zero_seq_len', 'non_zero_seq_len',
       'quantile025_week', 'max_weekday_demand', 'avg_price',
       'cheaper_than_usual',
       'demand__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"var"__maxlag_40',
       'demand__ar_coefficient__k_10__coeff_3',
       'demand__energy_ratio_by_chunks__num_segments_10__segment_focus_0',
       'demand__energy_ratio_by_chunks__num_segments_10__segment_focus_1',
       'demand__energy_ratio_by_chunks__num_segments_10__segment_focus_2',
       'demand__energy_ratio_by_chunks__num_segments_10__segment_focus_3',
       '

In [0]:
for column in X_val.columns:
 if X_val[column].dtype.name == 'category':
  dummyCols=pd.get_dummies(X_val[column])
  X_val=X_val.join(dummyCols)
  del X_val[column]

In [0]:
for column in X_train.columns:
 if X_train[column].dtype.name == 'category':
  dummyCols=pd.get_dummies(X_train[column])
  X_train=X_train.join(dummyCols)
  del X_train[column]

In [0]:
X_val = X_val.fillna(0)

In [0]:
X_train = X_train.dropna()

In [0]:
y_train = X_train['demand']
y_val = X_val['demand']
del X_train['demand']
del X_val['demand']

In [17]:
for feature in X_train.columns:
    print("{}: {}".format(feature, X_train[feature].corr(y_train)))

month: -0.005331694632323964
year: 0.00602476206448677
snap_WI: 0.04677626410824542
days_from_start: 0.004378053822540438
start_date_from_start: -0.02268596489262877
sell_price: -0.16807703617770417
lag_28: 0.5784407577671664
lag_29: 0.5661573644994532
lag_30: 0.5651533000667964
lag_31: 0.563769764621623
lag_32: 0.5533464205712468
lag_33: 0.5491572710858105
lag_34: 0.5528994652686361
lag_35: 0.5484823804133633
rmean_28_3: 0.6383461138395076
rmean_28_7: 0.6679982503163494
ADI: -0.1847997646336234
CV2: 0.17971717721806027
avg_weekday_demand: 0.6610092145325063
avg_demand: 0.6554178429515397
max_demand: 0.5434951921252689
std_demand: 0.6064130928759729
zero_seq_len: -0.07732519814595432
non_zero_seq_len: 0.5170988900343764
quantile025_week: 0.4606684628598245
max_weekday_demand: 0.5691419770604416
avg_price: -0.1678535454624901
cheaper_than_usual: 0.04361777909577033
demand__agg_autocorrelation__f_agg_"mean"__maxlag_40: 0.2918033404843618
demand__agg_autocorrelation__f_agg_"median"__maxla

In [18]:
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_val)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % np.sqrt(mean_squared_error(y_val, y_pred)))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_val, y_pred))


Coefficients: 
 [ 8.22747662e-02  1.05544758e+00  2.88427138e-01 -2.80602107e-03
 -2.62836342e-05 -5.92897492e-02 -1.20373821e+00 -1.25467354e+00
 -1.24306351e+00 -4.25919631e-01 -4.48191175e-01 -4.53715153e-01
 -4.44227832e-01  4.22569848e-02  2.45319410e+00  3.46497237e+00
  1.79093233e-04 -1.77507331e-02  9.08488318e-01 -3.88794025e-01
  6.10042009e-04 -7.92505877e-03 -1.41136313e-05 -6.62048145e-05
 -5.04112008e-03  2.57105522e-04  5.85657295e-02  6.30484992e-02
 -2.77763041e-01  3.10211803e-01  6.39772365e-01  6.83356010e-03
 -8.74859872e-02 -5.47305778e-02 -2.36206765e-02 -2.31151263e-02
 -5.07151224e-02 -4.74170713e-02 -1.88779272e-02 -5.84149089e-02
 -3.35902849e-02  6.06047557e-02 -1.01101611e-06 -4.07842197e-03
  5.14959064e-03 -1.07116867e-03]
Mean squared error: 3.14
Coefficient of determination: 0.59


In [0]:
X2.to_numpy()